# TP 2 de Statistique descriptive avec SAS

### Les formats

Un format est un *masque d'affichage des données*. Cet objet permet en SAS de faciliter la lecture des tables. 

Il existe deux types de format : les formats (aussi appelés les formats de lecture) et les informats (ou format d'écriture).

L'exemple typique d'utilisation de format est celui des formats de date. Par défaut, SAS stocke les dates sous la forme d'un entier qui correspond au nombre de jours entre la date et le 1er janvier 1960. Il s'agit donc d'une variable numérique. Il est très délicat pour un utilisateur de manipuler ces variables sous cette forme (à quoi correspond par exemple la date -4017?). Les formats de date permettent de changer l'affichage de la variable : on peut par exemple choisir d'afficher la date sous la forme Jour/Mois/Année à l'aide du format DDMMYY. ou sous la forme MoisAnnée à l'aide du format MONYY.

Les formats ne s'appliquent pas uniquement sur les dates mais sur n'importe quelle variable. On peut par exemple vouloir afficher des numéros de départements sur trois positions en complétant par des zéros à gauche (001, 002 ...) ou afficher un nombre restreint de décimales.

Une liste des formats est disponible ici.

Pour appliquer un format à une variable, on utilisera l'instruction FORMAT dans une procédure ou dans une étape DATA.

Par exemple, prenons la table AIR de la bibliothèque SASHELP.

Il est possible de créer 

## Exercice 1 : Analyse univariée

Dans cet exercice, on considère la table *pauv12dep.sas7bdat*.

**Question 1** : Créez une bibliothèque (ou library) *source* faisant le lien avec le dossier TP2/Source. Vous prendrez soin d'utiliser une option d'instruction permettant la lecture seule.

Créez également une bibliothèque *resultat* qui va contenir les tables en résultat de ce TP.

In [ ]:
LIBNAME source "/folders/myfolders/Donnees_TP_SAS/TP2/source" ACCESS = READONLY;
LIBNAME resultat "/folders/myfolders/Donnees_TP_SAS/TP2/resultat";

**Question 2** : créez une copie de la table RP12_d35 dans la bibliothèque *resultat*.

In [ ]:
PROC CONTENTS DATA = source.pauv12dep;
RUN;

## Exercice 2 : Analyse de l'âge et de la catégorie socioprofessionnelle des habitants d'Ille-et-Vilaine